In [39]:
import pandas as pd
import numpy as np
import datetime
import xlsxwriter
import warnings


warnings.filterwarnings('ignore')

df = pd.read_csv(r"D:\OneDrive - NITT\Custom_Download\new_abstimmung_1499.csv",
                 encoding='cp1252',
                 delimiter=';')

shop_number = df[df['KST']!=0]['KST'].unique()


def get_index(df, date, promised_amount, no_loopup_days=10):
    payment = df[['DATE', 'Haben']]
    offset = date + datetime.timedelta(days=no_loopup_days)

    small_df = payment[(payment['DATE'] <= offset) & (payment['DATE'] >= date)]
    small_df = small_df[~small_df['Haben'].isnull()]

    y = small_df[small_df['Haben'] == promised_amount]
    indx = y.index[0] if len(y) >= 1 else None
    return indx

pair_dict = dict()
for shop in sorted(shop_number):
    pair_list = []
    data = df[df['KST'].isin([shop]) | df['KST2'].isin([shop])]
    data['Soll'] = np.where(data['KST'] == shop, data['BETRAG'], np.nan)
    data['Haben'] = np.where(data['KST2'] == shop, data['BETRAG'], np.nan)

    data = data[['DATE', 'KST', 'KST2', 'Soll', 'Haben']]

    data['DATE'] = pd.to_datetime(data['DATE'], dayfirst=True, infer_datetime_format=True)

    data_clean = data[['DATE', 'Soll']][~data['Soll'].isnull()]
    for indx, row in data_clean.iterrows():
        indx_payment = get_index(data, row['DATE'], row['Soll'])
        Search_indx = sum(pair_list, ())

        if row['Soll'] < 0:
            indx_payment = data_clean[(data_clean['DATE'] == row['DATE']) & (data_clean['Soll'] == abs(row['Soll']))]
            if len(indx_payment):
                indx_payment = indx_payment.index[0]
                pair_list.append((indx_payment, indx))
        else:
            if indx_payment != None:
                if indx_payment not in Search_indx:
                    pair_list.append((indx, indx_payment))

    pair_dict[shop] = pair_list

indx_drop = sorted(sum(sum(pair_dict.values(), []), ()))

df_final = df.drop(indx_drop)

writer = pd.ExcelWriter(r'D:\OneDrive - NITT\Custom_Download\new_abstimmung_1499.xlsx', engine='xlsxwriter')

for shop in sorted(shop_number):
    shop_data = df_final[df_final['KST'].isin([shop]) | df_final['KST2'].isin([shop])]
    shop_data['Soll'] = np.where(shop_data['KST'] == shop, shop_data['BETRAG'], np.nan)
    shop_data['Haben'] = np.where(shop_data['KST2'] == shop, shop_data['BETRAG'], np.nan)

    if len(shop_data)>=1:
        shop_data.to_excel(writer, sheet_name=str(shop),startcol=0,index=False)
writer.save()